In [101]:
os.chdir('/home/mike/Desktop/Word Embeddings')
import os
import pandas as pd
import pickle

os.chdir('./Text Processing')
from TextPrep import TextPrep

os.chdir('../Meta Data')
from key_words import key_words, key_synonyms
from stop_words import stop_words
os.chdir('..')

In [102]:
# load in data
meta_data = pd.read_csv('Meta Data/meta_data.csv')
tweet_df = pd.read_csv('Data/aggregated_tweets.csv')
# merge data with meta data
tweet_df = pd.merge(tweet_df, meta_data, how = 'inner', on = 'user_id')
tweet_df = tweet_df[tweet_df.party.isin(['R', 'D'])].reset_index(drop=True)

tweets = tweet_df['text']
labels = tweet_df['party']

# initialize parser
prep = TextPrep(stopwords = stop_words, key_words = key_words, key_synonyms = key_synonyms)

In [104]:
%%time
# Process text
tweets = [prep.twitter_preprocess(tweet) for tweet in tweets]

for i in range(len(tweets)):
    try:
        tweets[i] = prep.tag_keywords('trump', tweets[i], labels[i])
    except:
        print('failed at '+ str(i))

#tweets = [prep.spacy_lemmatizer(tweet) for tweet in tweets[0:100]]
tweets = prep.multi_lemmatizer(tweets, threads = 6)
tweets[0:5]

CPU times: user 5min 16s, sys: 22.2 s, total: 5min 38s
Wall time: 1min 50s


[['not',
  'long',
  'vote',
  'impeachment',
  'inquiry',
  'open',
  'transparent',
  'today',
  'h',
  're',
  'learn',
  'numerous',
  'closed',
  'deposition',
  'take',
  'place',
  'week',
  's',
  'surprising',
  'stoptheschiffshow'],
 ['look',
  'forward',
  'show',
  'repmarkgreen',
  'fl',
  'week',
  'amp',
  'introduce',
  'outstanding',
  'veteran',
  'community',
  'florida'],
 ['yes',
  'vote',
  'resolution',
  'give',
  'stamp',
  'approval',
  'process',
  'damage',
  'repair',
  'watch',
  'floor',
  'speech',
  'morning',
  'florida',
  'fl',
  'impeachmentinquiry'],
 ['house',
  'vote',
  'resolution',
  'formalize',
  'impeachmentinquiry',
  'trump_r',
  'damage',
  'clear',
  'democrat',
  'interested',
  'precedent',
  'process',
  'goal',
  'remove',
  'trump_r',
  'office',
  'period'],
 ['dc',
  'team',
  'happy',
  'help',
  'theuso',
  'afternoon',
  'total',
  'care',
  'package',
  'assemble',
  'deployed',
  'service',
  'member',
  'thank',
  'defend',

In [90]:
from gensim.models import Word2Vec

In [106]:
%%time
model = Word2Vec(tweets, window = 10)

CPU times: user 14.3 s, sys: 32.4 ms, total: 14.3 s
Wall time: 6.34 s


In [107]:
model['trump_r']

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.79283136, -3.7154915 ,  0.08634412, -0.6042187 ,  4.1088676 ,
        1.3701674 , -2.320049  ,  0.6001312 ,  1.8058817 , -1.0399908 ,
        0.5023474 ,  0.21205613, -2.4601932 , -0.8055681 ,  0.7982369 ,
       -1.3374193 ,  0.53301424,  1.4107492 , -1.7609164 , -1.7289532 ,
       -1.0723376 ,  0.48007962, -0.63902617,  1.8698964 , -0.24873242,
       -2.4741359 , -4.144759  ,  0.524034  , -0.08972266, -3.8466797 ,
        1.4974976 , -3.0499096 ,  1.0085543 , -0.05841605,  0.51457024,
        2.0496285 ,  0.6672105 ,  1.6625658 ,  0.8009476 , -0.17432192,
        0.2616174 , -1.154765  , -0.07201336,  1.4105059 ,  0.41660213,
       -0.82416683,  0.9838408 , -0.6091365 ,  3.5399601 ,  1.7292374 ,
        0.97638375,  1.4019309 , -1.201303  , -3.048656  , -1.3767214 ,
        1.6087347 ,  2.2810743 ,  1.6823213 , -0.9131003 , -0.6242315 ,
        1.1505653 ,  1.3358098 , -0.9933186 ,  0.4517339 ,  1.7934008 ,
       -2.4477975 ,  1.9618927 , -1.7996788 , -1.6243473 ,  0.26

In [108]:
model.most_similar(positive=['trump_r'])

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('president', 0.6234646439552307),
 ('obama', 0.558197021484375),
 ('obsession', 0.5475559234619141),
 ('volodymyr', 0.5462174415588379),
 ('vp', 0.5273219347000122),
 ('previous', 0.5213545560836792),
 ('joebiden', 0.5152336359024048),
 ('hunt', 0.5149193406105042),
 ('whitehouse', 0.5097262859344482),
 ('collusion', 0.5095348358154297)]

In [109]:
model.most_similar(positive=['trump_d'])

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('outrageous', 0.63286292552948),
 ('reckless', 0.628778338432312),
 ('president', 0.6254245042800903),
 ('disgraceful', 0.6203300356864929),
 ('hellbent', 0.6150377988815308),
 ('obama', 0.611612856388092),
 ('immoral', 0.609902024269104),
 ('completely', 0.6080020666122437),
 ('betrayal', 0.6045030355453491),
 ('corruption', 0.6023545265197754)]

In [113]:
model.similarity('trump_r', 'trump_d')

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.5077271

In [117]:
model.similarity('trump_r', 'impeach')

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.4339764

In [116]:
model.similarity('trump_d', 'impeach')

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.20758998